# Captura de Dados provenientes de Fóruns no Reddit

*   Desenvolvedora: Paula Jeniffer dos Santos Viriato
*   Projeto: NIHER - Non-invasive and Highly Explainable Recommendations
*   Data: 17 de abril de 2023



In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!pip install praw

In [ ]:
import praw
import json
from IPython import display
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA
import nltk
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.tokenize import word_tokenize
import re

In [ ]:
with open("credentials.json", 'r') as jsonfile:
    credentials = json.load(jsonfile)

client_id = credentials["client_id"]
client_secret = credentials["client_secret"]
user_agent = "Comment Extraction (by u/"+credentials["username"]+")"
username = credentials["username"]
reddit = praw.Reddit(client_id=client_id, client_secret=client_secret, user_agent=user_agent, username=username, check_for_async=False)

In [ ]:
def recursive_coments (submission):
    recursive = []
    report_message = ", **UPVOTE** esse comentário.\n\nSe esta postagem não é adequada para o "
    submission.comments.replace_more(limit=None)
    if (len(submission.comments) > 0):
        for comment in submission.comments:
            if (report_message not in comment.body):
                try:
                    result = {}
                    result["body"] = comment.body
                    for reply in comment.replies:
                        try:
                            reply_result = recursive_coments(reply)
                            if (len(reply_result) > 0):
                                if ("comments" not in result.keys): result["comments"] = []
                                result["comments"].append(recursive_coments(reply))
                        except: continue
                    recursive.append(result)
                except: continue
    return recursive

In [10]:
topic = ''
posts = {}
post_topic = reddit.subreddit(topic)
count = 0
pack = 0
for post in post_topic.new(limit=10000):
    posts[post.id] = {}
    posts[post.id]["title"] = post.title
    posts[post.id]["selftext"] = post.selftext
    posts[post.id]["num_comments"] = post.num_comments
    try:
        comm_url = 'https://www.reddit.com/r/'+post.subreddit.display_name+'/comments/'+post.id+'/'
        comm_result = recursive_coments(reddit.submission(url=comm_url))
        if (len(comm_result) > 0): posts[post.id]["comments"] = recursive_coments(reddit.submission(url=comm_url))
    except: continue

    count += 1
    if (count%10 == 0):
        with open("reddit_"+topic+"_"+str(pack)+".json", 'w') as jsonfile:
            json.dump(posts, jsonfile, indent=4)
    if (count%100 == 0): pack += 1